## Detecção de Insatisfação com Agências
**Objetivo**: Implementar um pipeline de ponta a ponta para identificar e "flaggar" reclamações de clientes que expressam insatisfação com o atendimento ou serviços em agências físicas. O modelo utiliza um sistema de regras, léxicos e scores, sem depender de LLMs ou APIs externas.

In [10]:
# !pip install pandas unidecode

In [11]:
import pandas as pd
import re
from unidecode import unidecode
import json
from itertools import permutations

In [12]:
# 1. Léxico de Regras (Termos e Pesos)
termos_regras = [
    {'seq': ['gerente', 'atendimento', 'ruim'], 'peso': 0.9},
    {'seq': ['manipulado', 'gerente', 'agencia'], 'peso': 0.9},
    {'seq': ['falta', 'de', 'contato'], 'peso': 0.8},
    {'seq': ['tentativas', 'contato', 'frustrada'], 'peso': 0.8},
    {'seq': ['nao', 'atende'], 'peso': 0.8},
    {'seq': ['porta', 'trancada'], 'peso': 0.7},
    {'seq': ['sem', 'internet'], 'peso': 0.7},
    {'seq': ['nao', 'consegue', 'emitir'], 'peso': 0.6},
    {'seq': ['horario', 'agendado', 'cancelado'], 'peso': 0.7},
    {'seq': ['tempo', 'de', 'espera'], 'peso': 0.6},
    {'seq': ['cartao', 'bloqueado'], 'peso': 0.8},
    {'seq': ['caixa', 'eletronico', 'erro'], 'peso': 0.7},
    {'seq': ['central', 'atendimento', 'automatica'], 'peso': 0.6},
    {'seq': ['sistema', 'online', 'caiu'], 'peso': 0.6},
    {'seq': ['site', 'fora', 'ar'], 'peso': 0.6},
    {'seq': ['demora', 'retorno'], 'peso': 0.7},
    {'seq': ['solucionou', 'meu', 'problema'], 'peso': 0.8},
    {'seq': ['funcionario', 'inexperiente'], 'peso': 0.6},
    {'seq': ['bloqueado', 'indevidamente'], 'peso': 0.8},
    {'seq': ['fraude', 'conta'], 'peso': 0.7},
]

# 2. Dicionário de Sinônimos
sinonimos = {
    'gerente':       ['gerencia', 'responsavel', 'funcionario'],
    'atendimento':   ['suporte', 'assistencia', 'apoio'],
    'ruim':          ['péssimo', 'descaso', 'insatisfatorio'],
    'contato':       ['comunicacao', 'interacao', 'conexao', 'ligacao'],
    'frustrada':     ['sem exito', 'sem sucesso', 'va'],
    'falta':         ['ausencia', 'carencia'],
    'trancada':      ['fechada', 'bloqueada'],
    'internet':      ['online', 'wifi', 'rede'],
    'emitir':        ['gerar', 'imprimir', 'extrair'],
    'cancelado':     ['remarcado', 'desmarcado'],
    'espera':        ['fila', 'aguardo'],
    'bloqueado':     ['travado', 'retido'],
    'erro':          ['falha', 'bug', 'pane'],
    'automatica':    ['automatico', 'robotica'],
    'caiu':          ['falhou', 'derrubou'],
    'fora':          ['indisponivel', 'offline'],
    'demora':        ['atraso', 'lentidao'],
    'solucionou':    ['resolveu', 'corrigiu'],
    'inexperiente':  ['novato', 'iniciante'],
    'indevidamente': ['injustamente', 'sem motivo'],
    'fraude':        ['golpe', 'estelionato'],
}

In [13]:
df_reclamacoes = pd.read_excel('reclamacoes.xlsx')
df_reclamacoes.head(2)

texto  label
0  Há dias tento contato com a agência e ninguém ...    NaN
1  O gerente da agência foi extremamente rude e n...    NaN

In [14]:
def limpar_texto(texto):
    """Converte para minúsculas, remove acentos e caracteres especiais."""
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = unidecode(texto)
    texto = re.sub(r'[^a-z\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def segmentar_sentencas(texto):
    """Divide o texto em sentenças. 
    NOTA: Esta é uma abordagem simples. Para casos mais complexos,
    bibliotecas como NLTK (`nltk.sent_tokenize`) são mais robustas."""
    return [s.strip() for s in texto.split('.') if s.strip()]

def tokenizar_sentenca(sentenca):
    """Divide uma sentença em palavras (tokens)."""
    return sentenca.split()

print("Texto Original:", df_reclamacoes['texto'].iloc[0])
texto_limpo = limpar_texto(df_reclamacoes['texto'].iloc[0])
print("Texto Limpo:", texto_limpo)
sentencas = segmentar_sentencas(texto_limpo)
print("Sentenças:", sentencas)
tokens = tokenizar_sentenca(sentencas[0])
print("Tokens da primeira sentença:", tokens)

Texto Original: Há dias tento contato com a agência e ninguém me atende, total descaso com o cliente.
Texto Limpo: ha dias tento contato com a agencia e ninguem me atende total descaso com o cliente
Sentenças: ['ha dias tento contato com a agencia e ninguem me atende total descaso com o cliente']
Tokens da primeira sentença: ['ha', 'dias', 'tento', 'contato', 'com', 'a', 'agencia', 'e', 'ninguem', 'me', 'atende', 'total', 'descaso', 'com', 'o', 'cliente']


In [15]:
def expandir_com_sinonimos(termo, sinonimos_dict):
    """Retorna uma lista contendo o termo e seus sinônimos."""
    # Adiciona o próprio termo à lista de sinônimos para busca
    lista_sinonimos = sinonimos_dict.get(termo, [])
    return [termo] + lista_sinonimos

def verificar_proximidade(sentenca_tokens, sequencia, sinonimos_dict, janela_palavras=10):
    """
    Verifica se todos os termos de uma sequência (ou seus sinônimos)
    aparecem na sentença dentro de uma janela de palavras.
    """
    posicoes_encontradas = []
    termos_para_encontrar = list(sequencia)

    for i, token in enumerate(sentenca_tokens):
        for termo in list(termos_para_encontrar):
            # Verifica se o token corresponde ao termo ou a um de seus sinônimos
            if token in expandir_com_sinonimos(termo, sinonimos_dict):
                posicoes_encontradas.append(i)
                termos_para_encontrar.remove(termo)
                break # Vai para o próximo token da sentença
    
    # Se nem todos os termos da sequência foram encontrados, retorna False
    if termos_para_encontrar:
        return False, []

    # Se todos foram encontrados, verifica a distância
    if not posicoes_encontradas or (max(posicoes_encontradas) - min(posicoes_encontradas) >= janela_palavras):
        return False, []
    
    return True, posicoes_encontradas


def calcular_score_reclamacao(texto_reclamacao, regras, sinonimos_dict, janela_palavras=10):
    """
    Pipeline completo para calcular o score de uma única reclamação.
    (Versão Simplificada SEM modificadores de negação/intensidade)
    """
    # ETAPA 2: Pré-processamento
    texto_limpo = limpar_texto(texto_reclamacao)
    sentencas = segmentar_sentencas(texto_limpo)
    
    # ETAPA 3: Filtragem Inicial
    palavras_chave_filtro = expandir_com_sinonimos('agencia', sinonimos_dict)
    if not any(palavra in texto_limpo for palavra in palavras_chave_filtro):
        return 0.0, "Rejeitado no filtro inicial"

    scores_das_sentencas = []
    categorias_encontradas = set()

    # ETAPA 4: Pontuação por Sentença
    for sentenca in sentencas:
        sentenca_tokens = tokenizar_sentenca(sentenca)
        score_sentenca = 0.0
        
        for regra in regras:
            sequencia = regra['seq']
            peso = regra['peso']
            
            encontrado, posicoes = verificar_proximidade(sentenca_tokens, sequencia, sinonimos_dict, janela_palavras)
            
            if encontrado:
                # --- LÓGICA DE MODIFICADORES REMOVIDA DAQUI ---
                # Agora, simplesmente somamos o peso da regra.
                score_sentenca += peso
                
                categoria = regra.get('categoria')
                if categoria:
                    categorias_encontradas.add(categoria)

        scores_das_sentencas.append(score_sentenca)

    score_final = max(scores_das_sentencas) if scores_das_sentencas else 0.0
    
    info_categorias = ", ".join(sorted(list(categorias_encontradas))) if categorias_encontradas else ""
    
    return score_final, info_categorias

In [16]:
# Definindo o limiar para a flag
LIMIAR = 0.6

# Aplicando a função para calcular score e categorias
resultados = df_reclamacoes['texto'].apply(
    lambda texto: calcular_score_reclamacao(texto, termos_regras, sinonimos)
)

# Desempacotando os resultados em duas novas colunas
df_reclamacoes[['score_final', 'categorias']] = pd.DataFrame(resultados.tolist(), index=df_reclamacoes.index)

# Criando a flag final com base no limiar
df_reclamacoes['flag_agencia'] = df_reclamacoes['score_final'].apply(lambda x: 1 if x >= LIMIAR else 0)

In [17]:
display(df_reclamacoes.sort_values(by='score_final', ascending=False, ignore_index=True))

texto  label  score_final  \
0   Cartão de crédito bloqueado indevidamente, som...    NaN          1.6   
1   Fui à agência e o atendimento do gerente foi p...    NaN          0.9   
2   Estou há dias tentando contato com minha agênc...    NaN          0.8   
3   Falta de comunicação da agência, ninguém retor...    NaN          0.8   
4   O gerente da agência foi extremamente rude e n...    NaN          0.8   
5   Tentei visitar a agência para desbloquear meu ...    NaN          0.7   
6   Agência sem internet, não consigo emitir compr...    NaN          0.7   
7   Tive fraude na conta e a agência demorou dias ...    NaN          0.7   
8   Erro no caixa eletrônico da agência: prendeu m...    NaN          0.7   
9   Central de atendimento automática mal funciona...    NaN          0.6   
10  Visitei a agência hoje e fui atendido por um f...    NaN          0.6   
11  Experiência frustrada: agência sem gerente dis...    NaN          0.6   
12  Há dias tento contato com a agência e ninguém ...    NaN          0.0   
13  Ao contrário do que pensei, o atendimento na a...    NaN          0.0   
14  Tive um problema com meu cartão de crédito e o...    NaN          0.0   
15  A fila de espera na agência estava muito longa...    NaN          0.0   
16  O problema foi resolvido, mas o atendimento fo...    NaN          0.0   
17  O responsável pela minha conta na agência nunc...    NaN          0.0   
18  O aplicativo só funciona em wifi, bloqueando o...    NaN          0.0   
19  Não recebi o extrato mensal via app e a agênci...    NaN          0.0   
20  O sistema online caiu justo no dia que eu prec...    NaN          0.0   
21  Site do banco fora do ar no fim de semana, dei...    NaN          0.0   
22  Meu cartão chegou com defeito e a central de s...    NaN          0.0   
23  O aplicativo de internet banking apresentou er...    NaN          0.0   
24  O caixa da agência não aceitou meu depósito e ...    NaN          0.0   
25  A agência cancelou meu horário agendado sem av...    NaN          0.0   
26  Meu pedido de empréstimo na agência está parad...    NaN          0.0   
27  Essa empresa é horrível, o aplicativo é um lix...    NaN          0.0   

                     categorias  flag_agencia  
0                                           1  
1                                           1  
2                                           1  
3                                           1  
4                                           1  
5                                           1  
6                                           1  
7                                           1  
8                                           1  
9                                           1  
10                                          1  
11                                          1  
12                                          0  
13                                          0  
14  Rejeitado no filtro inicial             0  
15                                          0  
16  Rejeitado no filtro inicial             0  
17                                          0  
18  Rejeitado no filtro inicial             0  
19                                          0  
20  Rejeitado no filtro inicial             0  
21  Rejeitado no filtro inicial             0  
22  Rejeitado no filtro inicial             0  
23  Rejeitado no filtro inicial             0  
24                                          0  
25                                          0  
26                                          0  
27  Rejeitado no filtro inicial             0